In [6]:
import re
import pickle
import sklearn
import pandas as pd
import numpy as np

import holoviews as hv

import hvplot
import nltk 

import shutup;shutup.please()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from bokeh.io import output_notebook
output_notebook()

from hvplot import pandas
from pathlib import Path

#hv.extension("bokeh")

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

Loading BokehJS ...

In [7]:
data = pd.read_csv("sample_data.csv")

In [8]:
data

,doc_name,kwic,sentiment
0,115507,...for the physical and nutritional health of the nation and the opportunity that allotments pro...,positive
1,119218,...and context for the polit depend on the objectives pursued within each ical organization of s...,negative
2,119218,...connection to metabolism of nutrients diversification of local proximity improves traditional...,negative
3,119375,...and protect the environment of the cities hubbard and onumah 2001 asomani boateng 2002 bryld ...,negative
4,139077,...food close to settlements provides citizens to reach fresh and healthy food directly from the...,positive
...,...,...,...
23238,326198,...tomato bean are efficient poultry is important no green label local participation collaborati...,negative
23239,1945902,...2018 ieee smartworld ubiquitous intelligence computing advanced trusted computing scalable co...,negative
23240,2591973,...on line shopping 207 and especially the social networks 341 as part of their marketing strate...,positive
23241,5010702,...internet has facilitated protests and movements while fostering radical decentralization and ...,negative


In [16]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("english") 
stop_words = set(stopwords.words("english"))


def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE) #remove white space
    RE_TAGS = re.compile(r"<[^>]+>") #removes tags
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE) #keep only asscii character
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sitas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sitas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
clean_text("Python Is The best programing language in the , world")

'python best program languag world'

In [18]:
# Counting number of character in comment

char_count = []
for comment in data.kwic:
    char_count.append(len(comment))
    
len(char_count)
data['count'] = char_count

In [19]:
data

,doc_name,kwic,sentiment,count
0,115507,...for the physical and nutritional health of the nation and the opportunity that allotments pro...,positive,644
1,119218,...and context for the polit depend on the objectives pursued within each ical organization of s...,negative,652
2,119218,...connection to metabolism of nutrients diversification of local proximity improves traditional...,negative,752
3,119375,...and protect the environment of the cities hubbard and onumah 2001 asomani boateng 2002 bryld ...,negative,690
4,139077,...food close to settlements provides citizens to reach fresh and healthy food directly from the...,positive,638
...,...,...,...,...
23238,326198,...tomato bean are efficient poultry is important no green label local participation collaborati...,negative,557
23239,1945902,...2018 ieee smartworld ubiquitous intelligence computing advanced trusted computing scalable co...,negative,615
23240,2591973,...on line shopping 207 and especially the social networks 341 as part of their marketing strate...,positive,635
23241,5010702,...internet has facilitated protests and movements while fostering radical decentralization and ...,negative,800


In [ ]:
# # Removing all comment with less than 100 characters

# data = data.loc[data['count'] > 100,:]
# data.shape

In [20]:
# Adding a column with cleaned comment

clean_comment = []

for comment in data.kwic:
    clean_comment.append(clean_text(comment))
    
data['clean_comment'] = clean_comment

In [21]:
data

,doc_name,kwic,sentiment,count,clean_comment
0,115507,...for the physical and nutritional health of the nation and the opportunity that allotments pro...,positive,644,physic nutrit health nation opportun allot provid exercis fresh produc communiti local govern we...
1,119218,...and context for the polit depend on the objectives pursued within each ical organization of s...,negative,652,context polit depend object pursu within ical organ segreg communiti project studi document dive...
2,119218,...connection to metabolism of nutrients diversification of local proximity improves traditional...,negative,752,connect metabol nutrient diversif local proxim improv tradit practic water citi economi altern f...
3,119375,...and protect the environment of the cities hubbard and onumah 2001 asomani boateng 2002 bryld ...,negative,690,protect environ citi hubbard onumah asomani boateng bryld exampl develop agricultur land use act...
4,139077,...food close to settlements provides citizens to reach fresh and healthy food directly from the...,positive,638,food close settlement provid citizen reach fresh healthi food direct produc result food mile cov...
...,...,...,...,...,...
23238,326198,...tomato bean are efficient poultry is important no green label local participation collaborati...,negative,557,tomato bean effici poultri import green label local particip collabor central govern municip güd...
23239,1945902,...2018 ieee smartworld ubiquitous intelligence computing advanced trusted computing scalable co...,negative,615,ieee smartworld ubiquit intellig comput advanc trust comput scalabl comput communic cloud big da...
23240,2591973,...on line shopping 207 and especially the social networks 341 as part of their marketing strate...,positive,635,line shop especi social network part market strategi fact appear indic certain interest adapt cu...
23241,5010702,...internet has facilitated protests and movements while fostering radical decentralization and ...,negative,800,internet facilit protest movement foster radic decentr local control movement campaign might lab...


In [22]:
# Create binary class for positive and negative
data.loc[data["sentiment"] == "positive", "grade"] = 0
data.loc[data["sentiment"] == "negative", "grade"] = 1

data.grade.value_counts()

1.0    14667
0.0     8576
Name: grade, dtype: int64

In [23]:
data

,doc_name,kwic,sentiment,count,clean_comment,grade
0,115507,...for the physical and nutritional health of the nation and the opportunity that allotments pro...,positive,644,physic nutrit health nation opportun allot provid exercis fresh produc communiti local govern we...,0.0
1,119218,...and context for the polit depend on the objectives pursued within each ical organization of s...,negative,652,context polit depend object pursu within ical organ segreg communiti project studi document dive...,1.0
2,119218,...connection to metabolism of nutrients diversification of local proximity improves traditional...,negative,752,connect metabol nutrient diversif local proxim improv tradit practic water citi economi altern f...,1.0
3,119375,...and protect the environment of the cities hubbard and onumah 2001 asomani boateng 2002 bryld ...,negative,690,protect environ citi hubbard onumah asomani boateng bryld exampl develop agricultur land use act...,1.0
4,139077,...food close to settlements provides citizens to reach fresh and healthy food directly from the...,positive,638,food close settlement provid citizen reach fresh healthi food direct produc result food mile cov...,0.0
...,...,...,...,...,...,...
23238,326198,...tomato bean are efficient poultry is important no green label local participation collaborati...,negative,557,tomato bean effici poultri import green label local particip collabor central govern municip güd...,1.0
23239,1945902,...2018 ieee smartworld ubiquitous intelligence computing advanced trusted computing scalable co...,negative,615,ieee smartworld ubiquit intellig comput advanc trust comput scalabl comput communic cloud big da...,1.0
23240,2591973,...on line shopping 207 and especially the social networks 341 as part of their marketing strate...,positive,635,line shop especi social network part market strategi fact appear indic certain interest adapt cu...,0.0
23241,5010702,...internet has facilitated protests and movements while fostering radical decentralization and ...,negative,800,internet facilit protest movement foster radic decentr local control movement campaign might lab...,1.0


In [24]:
from bokeh.models import NumeralTickFormatter
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data["clean_comment"]).split()).value_counts()
word_freq[1:40].rename("Word frequency of most common words in comments").hvplot.bar(
    rot=45
).opts(width=700, height=400, yformatter=NumeralTickFormatter(format="0,0"))

:Bars   [index]   (Word frequency of most common words in comments)

In [25]:
# list most uncommon words
word_freq[0:].reset_index(name="freq").hvplot.table()

:Table   [index,freq]

In [26]:
word_freq[-10:].reset_index(name="freq").hvplot.table()

:Table   [index,freq]

In [27]:
# Distribution of ratings
data["grade"].value_counts(normalize=True).sort_index().hvplot.bar(title="Distribution of ratings")

:Bars   [index]   (grade)

In [28]:
"""
Compute unique word vector with frequencies
exclude very uncommon (<10 obsv.) and common (>=30%) words
use pairs of two words (ngram)
"""
vectorizer = TfidfVectorizer(
    analyzer="word", ngram_range=(1, 2), norm="l2")
vectorizer.fit(data["clean_comment"])

TfidfVectorizer(ngram_range=(1, 2))

In [30]:
# Vector representation of vocabulary
word_vector = pd.Series(vectorizer.vocabulary_).sample(5, random_state=1)
print(f"Unique word (ngram) vector extract:\n\n {word_vector}")

Unique word (ngram) vector extract:

 number architect    373295
realiti             446269
area simona          37499
voghera giudic      585518
citi produc          90075
dtype: int64


In [31]:
# Sample data - 25% of data to test set
train, test = train_test_split(data, random_state=1, test_size=0.25, shuffle=True)

X_train = train["clean_comment"]
Y_train = train["grade"]
X_test = test["clean_comment"]
Y_test = test["grade"]
print(X_train.shape)
print(X_test.shape)

(17432,)
(5811,)


In [32]:
# transform each sentence to numeric vector with tf-idf value as elements
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_train_vec.get_shape()

(17432, 608081)

In [33]:
# Compare original comment text with its numeric vector representation
print(f"Original sentence:\n{X_train[3:4].values}\n")
# Feature Matrix
features = pd.DataFrame(
    X_train_vec[3:4].toarray(), columns=vectorizer.get_feature_names()
)
nonempty_feat = features.loc[:, (features != 0).any(axis=0)]
print(f"Vector representation of sentence:\n {nonempty_feat}")

Original sentence:
['thornton barnett cloke clark malpass consum ethic articul subject space ethic consumpt antipod barthel parker ernstson food green space citi resili len garden urban environment movement urban studi battersbi beyond food desert find way speak urban food secur south africa geografiska annal seri human geographi bohstedt food riot polit']

Vector representation of sentence:
      africa  africa geografiska     annal  annal seri   antipod  \
0  0.059237            0.129555  0.098564     0.12241  0.091368   

   antipod barthel   articul  articul subject  barnett  barnett cloke  \
0         0.139251  0.094041         0.139251  0.11874       0.139251   

    barthel  barthel parker  battersbi  battersbi beyond    beyond  \
0  0.083454         0.10571   0.103019          0.135227  0.070916   

   beyond food  bohstedt  bohstedt food      citi  citi resili    clark  \
0     0.112032  0.144923       0.144923  0.025142     0.092978  0.09092   

   clark malpass     cloke  cl

In [35]:
# models to test
classifiers = [
    LogisticRegression(solver="sag", random_state=1),
    LinearSVC(random_state=1),
    #RandomForestClassifier(random_state=1),
    #XGBClassifier(random_state=1, use_label_encoder=False),
    #MLPClassifier( random_state=1,solver="adam",hidden_layer_sizes=(12, 12, 12),activation="relu",early_stopping=True,n_iter_no_change=1,),
]
# get names of the objects in list
names = [re.match(r"[^\(]+", name.__str__())[0] for name in classifiers]
print(f"Classifiers to test: {names}")

Classifiers to test: ['LogisticRegression', 'LinearSVC']


In [36]:
%%time
# test all classifiers and save pred. results on test data
results = {}
for name, clf in zip(names, classifiers):
    print(f"Training classifier: {name}")
    clf.fit(X_train_vec, Y_train)
    prediction = clf.predict(X_test_vec)
    report = sklearn.metrics.classification_report(Y_test, prediction)
    results[name] = report

Training classifier: LogisticRegression
Training classifier: LinearSVC
CPU times: total: 3.48 s
Wall time: 3.52 s


In [37]:
# Prediction results
for k, v in results.items():
    print(f"Results for {k}:")
    print(f"{v}\n")

Results for LogisticRegression:
              precision    recall  f1-score   support

         0.0       0.86      0.50      0.63      2081
         1.0       0.77      0.95      0.85      3730

    accuracy                           0.79      5811
   macro avg       0.82      0.73      0.74      5811
weighted avg       0.80      0.79      0.78      5811


Results for LinearSVC:
              precision    recall  f1-score   support

         0.0       0.82      0.72      0.77      2081
         1.0       0.85      0.91      0.88      3730

    accuracy                           0.84      5811
   macro avg       0.84      0.81      0.82      5811
weighted avg       0.84      0.84      0.84      5811




In [38]:
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

# define parameter space to test # runtime 35min
params = {
    "tfidf__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8, 0.2),
    "tfidf__min_df": np.arange(5, 100, 45),
}
pipe_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_clf.fit(X_train, Y_train)
# pickle.dump(pipe_clf, open("./clf_pipe.pck", "wb"))

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('svc', LinearSVC())]),
             n_jobs=-1,
             param_grid={'tfidf__max_df': array([0.3, 0.5, 0.7]),
                         'tfidf__min_df': array([ 5, 50, 95]),
                         'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             scoring='f1_macro')

In [39]:
print(pipe_clf.best_params_)

{'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 1)}


In [40]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("svc", LinearSVC())])

# define parameter space to test # runtime 19min
params = {
    "tfidf__ngram_range": [(1, 3)],
    "tfidf__max_df": [0.5],
    "tfidf__min_df": [5],
    "svc__C": np.arange(0.2, 1, 0.15),
}
pipe_svc_clf = GridSearchCV(pipe, params, n_jobs=-1, scoring="f1_macro")
pipe_svc_clf.fit(X_train, Y_train)
#pickle.dump(pipe_svc_clf, open("./pipe_svc_clf.pck", "wb"))

CPU times: total: 12.4 s
Wall time: 1min 7s


GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('svc', LinearSVC())]),
             n_jobs=-1,
             param_grid={'svc__C': array([0.2 , 0.35, 0.5 , 0.65, 0.8 , 0.95]),
                         'tfidf__max_df': [0.5], 'tfidf__min_df': [5],
                         'tfidf__ngram_range': [(1, 3)]},
             scoring='f1_macro')

In [41]:
best_params = pipe_clf.best_params_
print(best_params)

{'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 1)}


In [42]:
# run pipe with optimized parameters
pipe.set_params(**best_params).fit(X_train, Y_train)
pipe_pred = pipe.predict(X_test)
report = sklearn.metrics.classification_report(Y_test, pipe_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.80      0.82      0.81      2081
         1.0       0.90      0.88      0.89      3730

    accuracy                           0.86      5811
   macro avg       0.85      0.85      0.85      5811
weighted avg       0.86      0.86      0.86      5811



In [46]:
# Get confidence score for prediction
conf_score = pipe.decision_function(X_test)
# Get the Nth highest / lowest score
# high score indicates class 1 (bad), low score 0 (good)
score_neg = np.sort(conf_score)[-400]
score_pos = np.sort(conf_score)[100]
print(
    f"Threshold for negative rating: {score_neg}\nThreshold for positive rating: {score_pos}"
)

Threshold for negative rating: 2.4683301445692076
Threshold for positive rating: -1.0495018458035705


In [48]:
pd.options.display.max_colwidth = 800
# Predicted good but rated bad
test[["grade", "kwic"]][(Y_test != pipe_pred) & (conf_score <= score_pos)]

,grade,kwic
10344,1.0,...with original antagonism evolving into support and even initiation discussion such engagement stemmed from acknowledgement of the numer ous benefits of volunteer based community gardening projects in this paper we have explored the potential that integrating including improved neighbourhood appearance and stability per two distinct and largely independent forms of <green> <space> in <urban> haps the most fundamental benefits were economic as this allowed areas public parks and community gardens has for providing for a reduced reliance on public expenditure by transferring more ecosystem services to urban communities however discussion of responsibility to the community rosol 2010 rather than being the potential benefits needs...
18579,1.0,...how zfarming activities can serve as show which is reflected by the longing of city inhabitants to cases for education on food production famous examples become closer to the production of food again consumers include the science barge an urban greenhouse and edu ask for fresh local food with low <carbon> <footprints> more cational center that grows vegetables in new york city and transparency and closer involvement in the food produc is visited by school classes and thousands of visitors tion chain apa 2007 there is a growing interest throughout the year2 rooftop gardens and greenhouses can worldwide in establishing or re...


In [50]:
# Predicted bad but rated good
test[["grade", "kwic"]][(Y_test != pipe_pred) & ((conf_score >= score_neg))]

,grade,kwic


In [54]:
# Get new comments from website that were not included in original data
INPUT = [
    "increase in poverty cause increase in crimes",
    "if Crimes incresese, satisfaction in people decreases"
]

In [58]:
# Pre-Process comments as we did with train data
text = [clean_text(comment) for comment in INPUT]
text_out = " \n".join(text)
print(f"Input after pre-processing / cleaning:\n\n{text_out}")
# run comments through pipe: predict using our best model from above
predictions = pipe.predict(text)

Input after pre-processing / cleaning:

increas poverti caus increas crime 
crime increses satisfact peopl decreas


In [59]:
# Show comment and predicted Labels
predictions = pd.Series(predictions)
predictions = predictions.replace(0, "Senergy").replace(1, "Tradeoff")

pd.concat(
    [pd.Series(INPUT), predictions], axis="columns", keys=["comment", "prediction"]
)

,comment,prediction
0,increase in poverty cause increase in crimes,Tradeoff
1,"if Crimes incresese, satisfaction in people decreases",Tradeoff
